# 3. Find and Classify Cells

In [2]:
from predict import make_pred, post_processing, make_pred_dataframe
from dataset import TNBC2
import os
import random
import matplotlib.pyplot as plt
import config
from tqdm import tqdm
import numpy as np
c = config.Config()
n = 4

In [ ]:
tnbc = TNBC2()
model_name = 'unet_quip_10000'
stride = int(c.WIDTH*3/2)

for imid in tnbc.ids[:n]:
    path = tnbc.file_name(imid)
    img = tnbc.load_image(imid)
    pred = make_pred(path, model_name, stride)
    mask = post_processing(pred, 0.05)
    
    df_pred = make_pred_dataframe(imid, mask, img)
    pred_class = np.zeros_like(mask).astype(np.uint8)
    for _, row in df_pred[df_pred["image_id"] == imid].iterrows():
        pred_class[row["obj"]] = (row["class"]) * row["mask"]
        
    
    plt.figure(figsize=(40,20))
    plt.subplot(121)
    plt.imshow(img)
    plt.subplot(121)
    plt.imshow(pred, alpha=0.3)

## Predict and classify all TNBC

In [ ]:
import importlib
import dataset
from dataset import TNBC2
import keras.backend as K
importlib.reload(dataset)

tnbc = TNBC2()

dfs = []
for imid in tqdm(tnbc.ids):
    mask = post_processing(make_pred(tnbc.file_name(imid), 'unet_quip_ds', int(3*c.HEIGHT/2)))
    try:
        dfs.append(make_pred_dataframe(imid, mask, tnbc.load_image(imid)))
    except KeyError:
        continue
    K.clear_session()

df_pat = make_patient_dataframe(pd.concat(dfs))
df_pat.to_pickle('../data/tnbc2/tnbc.pickle')

  0%|          | 2/541 [00:04<19:00,  2.12s/it]

In [ ]:
df_pat = make_patient_dataframe(pd.concat(dfs))
df_pat.to_pickle('../data/tnbc2/tnbc.pickle')